# AutoGluon: Porto Seguro (Classification) # 4

Based on: https://www.linkedin.com/learning/automl-build-production-ready-models-quickly/

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "roc_auc"  # specify your evaluation metric
label = "target" # name of target or label variable (just happens to be called "target" for this dataset)

## Import

In [2]:
# Import packages

from autogluon.tabular import TabularDataset
from autogluon.tabular import TabularPredictor

import numpy as np
import pandas as pd

In [3]:
# Import data

train_data = TabularDataset("porto_train.csv")
test_data = TabularDataset("porto_test.csv")

In [4]:
train_data.shape

(476170, 60)

In [5]:
test_data.shape

(119042, 59)

## Tidy

In [6]:
# Replace "-1" with missing value
train_data = train_data.replace(-1, np.nan)
test_data = test_data.replace(-1, np.nan)

## Transform

In [7]:
# Drop variable not used as predictors
train_data = train_data.drop(["id", "fold"], axis=1)
test_data = test_data.drop(["id"], axis=1)

In [8]:
# Convert categorical variables to "category" data type
cat_vars = [col for col in train_data.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [9]:
for col in cat_vars:
    test_data[col] = test_data[col].astype('category')
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_data[col] = train_data[col].astype('category')

In [10]:
# Create separate test data to demonstrate how to make predictions on new examples at inference time:
y_test = test_data[label]  # values to predict

## Visualize

(skipped)

## Model

In [11]:
%%time

# gets stuck at
# Fitting model: CatBoost_BAG_L1 ... Training model for up to 3304.05s of the 3304.04s of remaining time.
#     Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
# so removed from fit
#     refit_full='best'
# crashed so killed all "jobs" and retried

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data, 
    presets=['best_quality'], 
    time_limit=time_limit,
    refit_full='best'
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230713_130608/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230713_130608/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Gener

	6.96s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: NeuralNetTorch_BAG_L1_FULL ...
	48.07s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMLarge_BAG_L1_FULL ...
	3.23s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	115.31s	 = Training   runtime
Refit complete, total runtime = 194.52s
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230713_130608/")


CPU times: user 2h 59min 12s, sys: 3min 20s, total: 3h 2min 33s
Wall time: 33min 38s


In [12]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

In [13]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6369310298022336
Evaluations on test data:
{
    "roc_auc": 0.6369310298022336,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [14]:
%%time

# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

CPU times: user 26min 11s, sys: 22.4 s, total: 26min 33s
Wall time: 1min 9s


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.636931,0.643510,38.769062,130.260926,1544.048239,0.017611,0.089578,115.306657,2,True,14
1,CatBoost_BAG_L1,0.636831,0.641170,2.479347,2.088559,444.585129,2.479347,2.088559,444.585129,1,True,7
2,CatBoost_BAG_L1_FULL,0.636259,NaN,0.205614,NaN,70.289076,0.205614,NaN,70.289076,1,True,19
3,XGBoost_BAG_L1,0.635973,0.638350,7.267202,4.584709,13.095284,7.267202,4.584709,13.095284,1,True,11
4,WeightedEnsemble_L2_FULL,0.635818,NaN,11.269884,NaN,324.212488,0.016712,NaN,115.306657,2,True,25
5,XGBoost_BAG_L1_FULL,0.632122,NaN,0.885103,NaN,6.955310,0.885103,NaN,6.955310,1,True,22
6,NeuralNetFastAI_BAG_L1,0.631466,0.634535,10.838213,21.684743,391.536835,10.838213,21.684743,391.536835,1,True,10
7,LightGBMXT_BAG_L1,0.625950,0.632746,1.256751,2.937709,13.838835,1.256751,2.937709,13.838835,1,True,3
8,LightGBMLarge_BAG_L1,0.625450,0.631070,1.284936,1.627526,11.028637,1.284936,1.627526,11.028637,1,True,13
9,LightGBMXT_BAG_L1_FULL,0.625159,NaN,0.151336,NaN,2.312966,0.151336,NaN,2.312966,1,True,15


In [15]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [16]:
%%time

test_leaderboard = predictor.leaderboard(extra_info=True, silent=True)

CPU times: user 46.4 s, sys: 17.6 s, total: 1min 4s
Wall time: 41.9 s


In [18]:
test_leaderboard.T

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
model,WeightedEnsemble_L2,CatBoost_BAG_L1,XGBoost_BAG_L1,NeuralNetFastAI_BAG_L1,LightGBMXT_BAG_L1,LightGBM_BAG_L1,LightGBMLarge_BAG_L1,NeuralNetTorch_BAG_L1,RandomForestEntr_BAG_L1,ExtraTreesEntr_BAG_L1,...,RandomForestEntr_BAG_L1_FULL,ExtraTreesEntr_BAG_L1_FULL,XGBoost_BAG_L1_FULL,WeightedEnsemble_L2_FULL,NeuralNetTorch_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL,LightGBM_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL
score_val,0.64351,0.64117,0.63835,0.634535,0.632746,0.631292,0.63107,0.624779,0.603324,0.602843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pred_time_val,130.260926,2.088559,4.584709,21.684743,2.937709,1.393207,1.627526,4.37075,30.749873,31.333006,...,30.749873,31.333006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time,1544.048239,444.585129,13.095284,391.536835,13.838835,10.409811,11.028637,494.613053,16.569741,14.503656,...,16.569741,14.503656,6.95531,324.212488,48.070812,26.134961,2.273739,2.312966,3.234969,70.289076
pred_time_val_marginal,0.089578,2.088559,4.584709,21.684743,2.937709,1.393207,1.627526,4.37075,30.749873,31.333006,...,30.749873,31.333006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time_marginal,115.306657,444.585129,13.095284,391.536835,13.838835,10.409811,11.028637,494.613053,16.569741,14.503656,...,16.569741,14.503656,6.95531,115.306657,48.070812,26.134961,2.273739,2.312966,3.234969,70.289076
stack_level,2,1,1,1,1,1,1,1,1,1,...,1,1,1,2,1,1,1,1,1,1
can_infer,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
fit_order,14,7,11,10,3,4,13,12,6,9,...,18,20,22,25,23,21,16,15,24,19
num_features,10,57,57,57,57,57,57,57,57,57,...,57,57,57,10,57,57,57,57,57,57
